In [2]:
!pip install -qU pip transformers sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.11.0 requires tensorflow~=2.11.0, which is not installed.
tf-models-official 2.11.0 requires pyyaml<6.0,>=5.1, but you have pyyaml 6.0.1 which is incompatible.
awscli 1.27.50 requires botocore==1.29.50, but you have botocore 1.34.39 which is incompatible.
awscli 1.27.50 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.
awscli 1.27.50 requires s3transfer<0.7.0,>=0.6.0, but you have s3transfer 0.10.0 which is incompatible.


In [3]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import io
import numpy as np
import os

from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

2024-02-11 00:02:57.357037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-11 00:02:58.014490: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-02-11 00:02:58.019921: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-02-11 00:02:58.378463: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
region_name='us-west-2'

s3 = boto3.client('s3')
bucket_name = 'aai-540-final-data'

model_dir = 'models/'
tar_file = 'model.tar.gz'

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

model_data = f's3://{bucket_name}/{model_dir}{tar_file}'
tensorflow_version = '2.6.3'
transformers_version='4.17.0'
py_version = 'py38'
huggingface_model = HuggingFaceModel(model_data=model_data,
                                     role=role,
                                     transformers_version=transformers_version,
                                     tensorflow_version=tensorflow_version,
                                     py_version=py_version,
                                     entry_point='inference.py'
                                   )

In [5]:
endpoint_config_name = 'emotion-ai-endpoint'

In [16]:
import time

# An array of different text strings for each call
texts = [
    "I just finished deploying a sample endpoint",
    "Today is a beautiful day for machine learning",
    "Exploring AWS SageMaker capabilities",
    "Optimizing model performance with hyperparameter tuning",
    "Implementing real-time predictions with SageMaker",
    "Analyzing large datasets can be very insightful",
    "Deep learning models have transformed AI",
    "Natural Language Processing enables new types of applications",
    "Predictive analytics can improve business decisions",
    "Reinforcement learning opens up new possibilities in AI"
]

def test_avg_time():
    times = []

    for text in texts:
        start_time = time.time()  # Start timing
        response = predictor.predict({"text": text})
        end_time = time.time()  # End timing
        times.append(end_time - start_time)  # Calculate the time taken for this call
        print(f"Response: {response}, Time taken: {end_time - start_time:.4f} seconds")

    # Calculate average time taken per call
    average_time = sum(times) / len(times)
    print(f"Average time per call: {average_time:.4f} seconds")

In [17]:
predictor = huggingface_model.deploy(initial_instance_count=1,
                                    instance_type='ml.g4dn.xlarge',
                                    endpoint_name=endpoint_config_name)

-----!

In [18]:
test_avg_time()

Response: ['{"neutral": 0.9839199185371399, "happy": 0.006058720406144857, "surprised": 0.005671103484928608}', 'application/json'], Time taken: 0.3709 seconds
Response: ['{"affectionate": 0.981002688407898, "happy": 0.012137876823544502, "optimistic": 0.0026838884223252535}', 'application/json'], Time taken: 0.2001 seconds
Response: ['{"neutral": 0.5749428272247314, "surprised": 0.40837612748146057, "happy": 0.010804630815982819}', 'application/json'], Time taken: 0.1721 seconds
Response: ['{"neutral": 0.9606403112411499, "surprised": 0.025737300515174866, "happy": 0.00609239749610424}', 'application/json'], Time taken: 0.1798 seconds
Response: ['{"neutral": 0.9815727472305298, "anger": 0.007424382958561182, "happy": 0.005755649879574776}', 'application/json'], Time taken: 0.1815 seconds
Response: ['{"neutral": 0.4026090204715729, "surprised": 0.19213369488716125, "happy": 0.17850379645824432}', 'application/json'], Time taken: 0.1826 seconds
Response: ['{"surprised": 0.62533491849899

In [19]:
predictor.delete_endpoint()

In [21]:
predictor = huggingface_model.deploy(initial_instance_count=1,
                                    instance_type='ml.c5.large',
                                    endpoint_name=endpoint_config_name)

-----!

In [22]:
test_avg_time()

Response: ['{"neutral": 0.9839199185371399, "happy": 0.006058720406144857, "surprised": 0.005671106278896332}', 'application/json'], Time taken: 0.5310 seconds
Response: ['{"affectionate": 0.981002688407898, "happy": 0.012137853540480137, "optimistic": 0.002683882135897875}', 'application/json'], Time taken: 0.3460 seconds
Response: ['{"neutral": 0.5749432444572449, "surprised": 0.4083758294582367, "happy": 0.010804599151015282}', 'application/json'], Time taken: 0.2209 seconds
Response: ['{"neutral": 0.9606404304504395, "surprised": 0.025737248361110687, "happy": 0.006092398427426815}', 'application/json'], Time taken: 0.2143 seconds
Response: ['{"neutral": 0.9815727472305298, "anger": 0.0074243973940610886, "happy": 0.005755661055445671}', 'application/json'], Time taken: 0.2143 seconds
Response: ['{"neutral": 0.40260910987854004, "surprised": 0.19213354587554932, "happy": 0.1785043478012085}', 'application/json'], Time taken: 0.2237 seconds
Response: ['{"surprised": 0.62533485889434

In [14]:
predictor.delete_endpoint()

In [ ]:
predictor = huggingface_model.deploy(initial_instance_count=1,
                                    instance_type='ml.t3.medium',
                                    endpoint_name=endpoint_config_name)

In [ ]:
test_avg_time()

In [ ]:
predictor.delete_endpoint()